In [ ]:
import pandas as pd
import xgboost as xgb
import shap
import numpy as np
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import xgboost as xgb 
from sklearn.linear_model import  LogisticRegression, LinearRegression
from sklearn import metrics
import matplotlib.pyplot as plt 
import mlxtend
import imblearn 
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer
from sklearn.model_selection import  cross_validate, cross_val_score
import warnings
warnings.filterwarnings('ignore')

### Comment (Hyperopt parameter)
- optimal estimator 126, max_depth rf = 8 (rf) 
- optimal estimator 228 , maxdepth = 8 (xgb)
* Optimized model do not perform any better than default 

In [ ]:
#imputed 
traindata = pd.read_csv('../doseprediction/data/sl_train.csv')
testdata = pd.read_csv('../doseprediction/data/sl_val.csv')
print(traindata.shape)
print(testdata.shape)

In [ ]:
keyfeatures = ['Gender','Ventilator', 'diabetes', 'metastatic_cancer', 'qsofa_sysbp_score',
       'qsofa_resprate_score', 'qsofa_gcs_score', 
        'Age', 'Weight', 'Height', 'bmi', 'ANION_GAP',
       'APTT', 'Albumine', 'Art_BE', 'Art_PH',  'Bicarbonaat',
       'Calcium', 'Chloride', 'DIA', 'FiO2', 'Glucose', 'HB', 'HeartRate',
       'Ion_Ca', 'Kalium', 'LEU', 'Lactate', 'MAP', 'Magnesium', 'Natrium',
       'PF_ratio', 'PaCO2', 'PaO2', 'RespRate', 'SYS', 'Shock_Index',
       'Sirs_score', 'Sofa_score', 'Temp', 'Trombo', 'elixhauser',
       'elixhauser_hospital', 'qsofa', 'mingcs', 'lods', 
       'SpO2', 'Ureum', 'Creat', 'ALAT',
       'ASAT', 'Bili', 'INR' ]# 'blood_culture_positive','race_white', 'race_black', 'race_hispanic', 'race_other']

fluidfeatures = ['cumm_fluid_balance','total_IV_prev', 'max_VP_prev', 
                            'Running_total_UP', 'total_UP']

target = ['max_VP', 'total_IV', 'discrete_action', 'discrete_VP', 
          'discrete_IV']

In [ ]:
#training predictors
X_train = traindata[keyfeatures]
X_test = testdata[keyfeatures]
X_train_extra =traindata[keyfeatures+fluidfeatures]
X_test_extra =testdata[keyfeatures+fluidfeatures]

# training targets 
Y_train = traindata[target]
Y_test = testdata[target]
#binarize VP as it is highly imbalance 
Y_train['binary_VP']=  np.where(Y_train.discrete_VP==0, 0, 1)
Y_test = testdata[target]
Y_test['binary_VP']=  np.where(Y_test.discrete_VP==0, 0, 1)

# Y all  for visualization 
Y = pd.concat([Y_train, Y_test])
X = pd.concat([X_train, X_test])
X_extra = pd.concat([X_train_extra, X_test_extra])

In [ ]:
def eval_classification(model, x_train, y_train, x_test, y_test, modelname):
    model.fit(x_train, y_train)
    predict = model.predict(x_test)
    predict_proba = model.predict_proba(x_test)
    f1 = metrics.f1_score(y_test, predict)
    accuracy = metrics.balanced_accuracy_score(y_test, predict)
    gmean = imblearn.metrics.geometric_mean_score(y_test, predict)
    logloss = metrics.log_loss(y_test, predict_proba)
    report = metrics.classification_report(y_test, predict)
    precision = metrics.precision_score(y_test, predict)
    recall = metrics.recall_score(y_test, predict)
    rocauc = metrics.roc_auc_score(y_test, predict_proba[:,1])
    regular_accuracy = metrics.accuracy_score(y_test, predict)
    eval = {'Model' : modelname,
            'F1' : f1,
            'Balanced_Accuracy' : accuracy,
            'Geo_Mean' : gmean,
            'log loss' : logloss,
            'Precision' : precision,
            'Recall' : recall,
            'Regular_Accuracy' : regular_accuracy,
            'AUCROC' : rocauc
            }
    #print(eval)
    return model, pd.DataFrame(eval, index=[0])

performance = pd.DataFrame()

In [ ]:
#model logistic regression
modelname = 'LR-base'
model1 = LogisticRegression(random_state=112)
model1, per= eval_classification(model1, X_train, Y_train.binary_VP, X_test, Y_test.binary_VP, modelname )
performance= performance.append(per)

In [ ]:
#model Random forest Base
modelname = 'RF-base'
model2 = RandomForestClassifier(random_state=112, n_jobs=-1)
model2, per= eval_classification(model2, X_train, Y_train.binary_VP, X_test, Y_test.binary_VP, modelname )
performance= performance.append(per)

In [ ]:
#model XGB Base
modelname = 'XGB-base'
model3 = xgb.XGBClassifier(random_state=112, n_jobs=-1, eval_metric='logloss')
model3, per= eval_classification(model3, X_train, Y_train.binary_VP, X_test, Y_test.binary_VP, modelname )
performance= performance.append(per)

In [ ]:
#model LR base+
modelname = 'LR-base+'
model4 = LogisticRegression(random_state=112)
model4, per= eval_classification(model4, X_train_extra, Y_train.binary_VP, X_test_extra, Y_test.binary_VP, modelname )
performance= performance.append(per)

In [ ]:
#model Random forest Base+
modelname = 'RF-base+'
model5 = RandomForestClassifier(random_state=112, n_jobs=-1)
model5, per= eval_classification(model5, X_train_extra, Y_train.binary_VP, X_test_extra, Y_test.binary_VP, modelname )
performance= performance.append(per)

In [ ]:
#model XGB base +
modelname = 'XGB-base+'
model6 = xgb.XGBClassifier(random_state=112, n_jobs=-1, eval_metric='logloss')
model6, per= eval_classification(model6, X_train_extra, Y_train.binary_VP, X_test_extra, Y_test.binary_VP, modelname )
performance= performance.append(per)

In [ ]:
performance

#### Significance test 

In [ ]:
from mlxtend.evaluate import paired_ttest_5x2cv

### Compare RF-base + (model 5) and XGB base + (model 6)  
t,p = paired_ttest_5x2cv(estimator1=model5,
                          estimator2=model6,
                          X=X_extra, y=Y.binary_VP,
                          random_seed=112, 
                          scoring=make_scorer(imblearn.metrics.geometric_mean_score))

print('stat:', t)
print('p-value:', p)

# interpret the p-value
alpha = 0.05
if p > alpha:
    print('Same proportions of errors (fail to reject H0), \
    and may conclude that the performance of the two algorithms is not significantly different.')
else:
    print('Different proportions of errors (reject H0), model are significantly different')
        



In [ ]:
### Compare LR-base + (model 4) and XGB base + (model 6)  
t,p = paired_ttest_5x2cv(estimator1=model4,
                          estimator2=model6,
                          X=X_extra, y=Y.binary_VP,
                          random_seed=112, 
                          scoring=make_scorer(imblearn.metrics.geometric_mean_score))

print('stat:', t)
print('p-value:', p)

# interpret the p-value
alpha = 0.05
if p > alpha:
    print('Same proportions of errors (fail to reject H0), \
    and may conclude that the performance of the two algorithms is not significantly different.')
else:
    print('Different proportions of errors (reject H0), model are significantly different')

#### Macnemar test RF and XGB 

In [ ]:
from mlxtend.evaluate import mcnemar_table, mcnemar
print("McNemar:")
y_model5 = model5.predict(X_test_extra)
y_model6 = model6.predict(X_test_extra)

        # Calculate p value
tb = mcnemar_table(y_target=Y_test.binary_VP, 
                          y_model1=y_model5, 
                          y_model2=y_model6)
        
chi2, p = mcnemar(ary=tb, exact=True)

print('chi-squared:', chi2)
print('p-value:', p)
# interpret the p-value
alpha = 0.05
if p > alpha:
    print('Same proportions of errors (fail to reject H0)-model are same')
else:
    print('Different proportions of errors (reject H0), significant difference in model performance')


In [ ]:
#RF confusion matrix
metrics.confusion_matrix(Y_test.binary_VP, y_model5)

In [ ]:
#XGB confusion matrix
metrics.confusion_matrix(Y_test.binary_VP, y_model6)

In [ ]:
#Original data 
metrics.confusion_matrix(Y_test.binary_VP, Y_test.binary_VP)

In [ ]:
model6_cls_report = imblearn.metrics.classification_report_imbalanced(Y_test.binary_VP, y_model6)
print(model6_cls_report)

In [ ]:
model5_cls_report = imblearn.metrics.classification_report_imbalanced(Y_test.binary_VP, y_model5)
print(model5_cls_report)

### Feature importance and shap value 

In [ ]:
def get_feature_importance(model, x_test):
    names=x_test.columns
    #Create arrays from feature importance and feature names
    feature_importance = np.array(model.feature_importances_)
    feature_names = np.array(names)

#Create a DataFrame using a Dictionary
    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)

#Sort the DataFrame in order decreasing feature importance
    fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)
    return fi_df

def shapanalysis(model, x_test):
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(x_test)
    return shap_values

In [ ]:
# Only for XGBoost -base + model 
fi_model6 = get_feature_importance(model6, X_test_extra)
shap_value = shapanalysis(model6, X_test_extra)

In [ ]:
#Plot feature importance 
plt.figure(figsize=(10, 8))
sns.barplot(x=fi_model6['feature_importance'][0:15], 
y=fi_model6['feature_names'][0:15])
plt.show()
shap.summary_plot(shap_value, X_test_extra , max_display=15, show=False)

In [ ]:
#### ROC Curve 
y_pred = model1.predict_proba(X_test)[:, 1]
fpr, tpr, _ = metrics.roc_curve(Y_test.binary_VP, y_pred)
auc = round(metrics.roc_auc_score(Y_test.binary_VP, y_pred), 4)
plt.plot(fpr,tpr,label="LR-base (basemodel), AUC="+str(auc))


y_pred = model2.predict_proba(X_test)[:, 1]
fpr, tpr, _ = metrics.roc_curve(Y_test.binary_VP, y_pred)
auc = round(metrics.roc_auc_score(Y_test.binary_VP, y_pred), 4)
plt.plot(fpr,tpr,label="RF-base, AUC="+str(auc))

y_pred = model3.predict_proba(X_test)[:, 1]
fpr, tpr, _ = metrics.roc_curve(Y_test.binary_VP, y_pred)
auc = round(metrics.roc_auc_score(Y_test.binary_VP, y_pred), 4)
plt.plot(fpr,tpr,label="XGB-base, AUC="+str(auc))

y_pred = model4.predict_proba(X_test_extra)[:, 1]
fpr, tpr, _ = metrics.roc_curve(Y_test.binary_VP, y_pred)
auc = round(metrics.roc_auc_score(Y_test.binary_VP, y_pred), 4)
plt.plot(fpr,tpr,label="LR-base+, AUC="+str(auc))



y_pred = model5.predict_proba(X_test_extra)[:, 1]
fpr, tpr, _ = metrics.roc_curve(Y_test.binary_VP, y_pred)
auc = round(metrics.roc_auc_score(Y_test.binary_VP, y_pred), 4)
plt.plot(fpr,tpr,label="RF-base+, AUC="+str(auc))

y_pred = model6.predict_proba(X_test_extra)[:, 1]
fpr, tpr, _ = metrics.roc_curve(Y_test.binary_VP, y_pred)
auc = round(metrics.roc_auc_score(Y_test.binary_VP, y_pred), 4)
plt.plot(fpr,tpr,label="XGB-base+, AUC="+str(auc))
plt.legend()
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

In [ ]:
import joblib
VP_models = {'LR-base' : model1,
              'RF-base' : model2,
              'XGB-base' : model3,
              'LR-base+': model4,
              'RF-base+': model5,
              'XGB-base+' : model6
              }
VP_feature_imp = {'XGB-base+_shape' : shap_value,
                  'XGB-base+_fi' : fi_model6}

performance.to_csv('../doseprediction/Final_VP_model_Performances.csv')
joblib.dump(VP_models, '../doseprediction/Final_VP_models.pkl')
joblib.dump(VP_feature_imp,  '../doseprediction/Final_VP_Featureimp.pkl')


### Cross validation set up 

In [ ]:
cv_results = pd.DataFrame()
scoring = {'f1_score': make_scorer(metrics.f1_score, greater_is_better=True),
           'log_loss' : make_scorer(metrics.log_loss),
           'g_mean' : make_scorer(imblearn.metrics.geometric_mean_score, greater_is_better=True),
           'b_accuracy' : make_scorer(metrics.balanced_accuracy_score, greater_is_better=True),
        'precision': make_scorer(metrics.precision_score, greater_is_better=True),
        'recall': make_scorer(metrics.recall_score, greater_is_better=True),
        'reg_accuracy': make_scorer(metrics.accuracy_score, greater_is_better=True)}

In [ ]:
#logistic base
modelname = 'LR-base'
lgfit = LogisticRegression()
lg_results = cross_validate(lgfit, X, Y.binary_VP, cv=5,
                         scoring=scoring)
lg_results = pd.DataFrame.from_dict(lg_results)
results = pd.DataFrame()
results['mean'] = lg_results.drop(['fit_time', 'score_time'], axis=1).mean()
results['se'] = lg_results.drop(['fit_time', 'score_time'], axis=1).sem()
results = results.T
results['model'] = modelname
#cv_results = pd.concat([cv_results, results])
cv_results =cv_results.append(results)


In [ ]:
#logistic base+
modelname = 'LR-base+'
lgfit = LogisticRegression()
lg_results = cross_validate(lgfit, X_extra, Y.binary_VP, cv=5,
                         scoring=scoring)
lg_results = pd.DataFrame.from_dict(lg_results)
results = pd.DataFrame()
results['mean'] = lg_results.drop(['fit_time', 'score_time'], axis=1).mean()
results['se'] = lg_results.drop(['fit_time', 'score_time'], axis=1).sem()
results= results.T
results['model'] = modelname
#cv_results = pd.concat([cv_results, results])
cv_results =cv_results.append(results)

In [ ]:
#rf-base
modelname = 'RF-base'
lgfit = RandomForestClassifier(n_jobs=-1)
lg_results = cross_validate(lgfit, X, Y.binary_VP, cv=5,
                         scoring=scoring)
lg_results = pd.DataFrame.from_dict(lg_results)
results = pd.DataFrame()
results['mean'] = lg_results.drop(['fit_time', 'score_time'], axis=1).mean()
results['se'] = lg_results.drop(['fit_time', 'score_time'], axis=1).sem()
results= results.T
results['model'] = modelname
cv_results =cv_results.append(results)

In [ ]:
#rf-base+
modelname = 'RF-base+'
lgfit = RandomForestClassifier(n_jobs=-1)
lg_results = cross_validate(lgfit, X_extra, Y.binary_VP, cv=5,
                         scoring=scoring)
lg_results = pd.DataFrame.from_dict(lg_results)
results = pd.DataFrame()
results['mean'] = lg_results.drop(['fit_time', 'score_time'], axis=1).mean()
results['se'] = lg_results.drop(['fit_time', 'score_time'], axis=1).sem()
results= results.T
results['model'] = modelname
cv_results =cv_results.append(results)

In [ ]:
#xgb-base
modelname = 'XGB-base'
lgfit = xgb.XGBClassifier(n_jobs=-1, eval_metric='logloss')
lg_results = cross_validate(lgfit, X, Y.binary_VP, cv=5,
                         scoring=scoring)
lg_results = pd.DataFrame.from_dict(lg_results)
results = pd.DataFrame()
results['mean'] = lg_results.drop(['fit_time', 'score_time'], axis=1).mean()
results['se'] = lg_results.drop(['fit_time', 'score_time'], axis=1).sem()
results= results.T
results['model'] = modelname
cv_results =cv_results.append(results)

In [ ]:
modelname = 'XGB-base+'
lgfit = xgb.XGBClassifier(n_jobs=-1, eval_metric='logloss')
lg_results = cross_validate(lgfit, X_extra, Y.binary_VP, cv=5,
                         scoring=scoring)
lg_results = pd.DataFrame.from_dict(lg_results)
results = pd.DataFrame()
results['mean'] = lg_results.drop(['fit_time', 'score_time'], axis=1).mean()
results['se'] = lg_results.drop(['fit_time', 'score_time'], axis=1).sem()
results= results.T
results['model'] = modelname
cv_results =cv_results.append(results)

In [ ]:
cv_results

In [ ]:
### Save VP Cross validation results
cv_results.to_csv('../doseprediction/Final_VP_CV_performance.csv')

### Some stats about the Data

In [ ]:
print('total features in base model: ', X_train.shape[1])
print('total features in base+ model: ', X_train_extra.shape[1])
